CE080_Vedant_Parikh

Machine Learning

Lab : 07

In [1]:
from __future__ import absolute_import, division, print_function
import tensorflow as tf
import numpy as np

In [2]:
from tensorflow.keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = np.array(x_train, np.float32), np.array(x_test, np.float32)
num_features=784
print(x_train.shape)
x_train, x_test = x_train.reshape([-1, num_features]), x_test.reshape([-1, num_features])
print(x_train.shape)
x_train, x_test = x_train / 255., x_test / 255.

11490434/11490434 [==============================] - 0s 0us/step
(60000, 28, 28)
(60000, 784)


In [3]:
num_classes = 10 # 0 to 9 digits
num_features = 784 # 28*28
learning_rate = 0.01
training_steps = 1000
batch_size = 256
display_step = 50

In [4]:
train_data=tf.data.Dataset.from_tensor_slices((x_train,y_train))
train_data=train_data.repeat().shuffle(5000).batch(batch_size).prefetch(1)
print(train_data)

<PrefetchDataset element_spec=(TensorSpec(shape=(None, 784), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.uint8, name=None))>


In [5]:
W = tf.Variable(tf.ones([num_features, num_classes]), name="weight")
print(W)
b = tf.Variable(tf.zeros([num_classes]), name="bias")
print(b)

<tf.Variable 'weight:0' shape=(784, 10) dtype=float32, numpy=
array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]], dtype=float32)>
<tf.Variable 'bias:0' shape=(10,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>


In [6]:
def logistic_regression(x):
    sm=tf.nn.softmax(tf.matmul(x, W) + b)
    return sm
def cross_entropy(y_pred, y_true):
    y_true = tf.one_hot(y_true, depth=num_classes)
    y_pred = tf.clip_by_value(y_pred, 1e-9, 1.)
    entropy=tf.reduce_mean(-tf.reduce_sum(y_true * tf.math.log(y_pred)))
    return entropy

In [7]:
def accuracy(y_pred, y_true):
    correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.cast(y_true, tf.int64))
    acc=tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    return acc
optimizer = tf.optimizers.SGD(learning_rate)

In [8]:
def run_optimization(x, y):
    with tf.GradientTape() as g:
        pred = logistic_regression(x)
        loss = cross_entropy(pred, y)
    gradients = g.gradient(loss, [W, b])
    optimizer.apply_gradients(zip(gradients, [W, b]))

In [9]:
for step, (batch_x, batch_y) in enumerate(train_data.take(training_steps), 1):
    run_optimization(batch_x, batch_y)
    if step % display_step == 0:
        pred = logistic_regression(batch_x)
        loss = cross_entropy(pred, batch_y)
        acc = accuracy(pred, batch_y)
        print("step: %i, loss: %f, accuracy: %f" % (step, loss, acc))

step: 50, loss: 1015.497681, accuracy: 0.675781
step: 100, loss: 593.160645, accuracy: 0.859375
step: 150, loss: 741.466309, accuracy: 0.796875
step: 200, loss: 745.540527, accuracy: 0.718750
step: 250, loss: 406.003235, accuracy: 0.898438
step: 300, loss: 600.898132, accuracy: 0.804688
step: 350, loss: 520.850525, accuracy: 0.863281
step: 400, loss: 799.557922, accuracy: 0.757812
step: 450, loss: 561.946411, accuracy: 0.851562
step: 500, loss: 508.219452, accuracy: 0.863281
step: 550, loss: 516.673340, accuracy: 0.851562
step: 600, loss: 623.716736, accuracy: 0.769531
step: 650, loss: 565.562866, accuracy: 0.847656
step: 700, loss: 621.072632, accuracy: 0.843750
step: 750, loss: 652.646973, accuracy: 0.824219
step: 800, loss: 505.824585, accuracy: 0.859375
step: 850, loss: 630.961914, accuracy: 0.785156
step: 900, loss: 666.335938, accuracy: 0.800781
step: 950, loss: 571.536316, accuracy: 0.855469
step: 1000, loss: 721.782532, accuracy: 0.820312


In [10]:
pred = logistic_regression(x_test)
print("Test Accuracy: %f" % accuracy(pred, y_test))

Test Accuracy: 0.834900
